# Monte Carlo Calculation of Pi

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/master/notebook/statistical-mechanics/monte_carlo_pi.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;"/>

## **Goals**

* Monte Carlo (MC) methods constitute a large category of computational algorithms and techniques, 
which rely on random number sampling to solve problems numerically. In this notebook
we introduce the Monte Carlo method via a simple simulation used to calculate the value of Pi.
* Understand the concept of the Monte Carlo method.
* Learn how to calculate the value of Pi via Monte Carlo simulation.
* Familiarize yourself with the Quasi-Monte Carlo method.
* Appreciate the effect of the number of samples on the accuracy of the results of MC calculations.


## **Background theory** 

[More on the background theory.](./theory/theory_monte_carlo_pi.ipynb)

## **Tasks and exercises**

1. What is meant by a uniformly distributed random number? 
    <details>
    <summary style="color: red">Solution</summary>
       If a discrete random number $x$ is distributed according to the uniform distribution, $x \sim \mathcal{U}\{a,b\}$, this means that the probability of $x$ taking on an integer value between the two integers $a$ and $b$ (both inclusive) is simply $1/n$ where $b-a+1=n$. For example, if $b=5$ and $a=2$, then $x$ may assume the values 2,3,4,5, each with probability $\frac{1}{5-2+1}=1/4$. In the case that $x$ is continuous, uniformly distributed takes on an analogous meaning. Now however, we must talk of $x$ assuming a value in some range $dx$. Then "x is continuously, uniformly distributed on the interval $(a,b)$, means that the probability of finding $x$ in some range $(x_0,x_0+dx)$ in that interval is simply $\frac{dx}{b-a}$. In other words, there is a uniform probability of finding $x$ at any point within the interval $(a,b)$.
    </details>

2. In this notebook, we are using pseudorandom numbers. Why is it that the generated numbers we use are not true random numbers?
    <details>
    <summary style="color: red">Solution</summary>
        In computational science, "random" numbers are generated by some algorithm. Ultimately, such algorithms are intrinsically deterministic: after having specified an input to the algorithm, referred to as a seed, the sequence of generated "random" numbers is fully determined. This seed is typically chosen in a way so as to try and introduce as much randomness into the simulation in possible. Examples include the current time, or physical quantities which possess some inherent noise like temperature, etc. So, in summary,
        we cannot have a truly random number generator for this notebook.
    </details>

3. How could one obtain a better estimate for the value of Pi from the simulation?
    <details>
    <summary style="color: red">Solution</summary>
        The accuracy of the simulation largely depends on the quality of the random numbers employed and the total number of samples used in the computation. Using the quasi-random number approach can improve the quality of the random numbers and thus convergence of the calculation. On the other hand, one can also improve the accuracy of the result by increasing the number of samples generated. Of course, the latter implies that longer simulation times shall be necessary and thus we have a time-accuracy trade-off.    </details>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Interactive visualization
(be patient, it might take a few seconds to load)

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets import Button, IntSlider, HBox, Checkbox, HTML
from matplotlib.animation import FuncAnimation

In [ ]:
Pi = np.pi

samples_inside = [];
samples_outside = [];

pi_x = [];
pi_y = [];

run_button = Button(description='Throw')
clear_button = Button(description='Clear')
num_coin = IntSlider(value=1, min=1, max=200, description='Number of coins:', style={'description_width': 'initial'})
use_quasi = Checkbox(value=False, description='Quasi-Monte Carlo')

label1 = HTML(value = f"<b><font color='blue' size=3>Number of inside coins:</b>")
label2 = HTML(value = f"<b><font color='red' size=3>Number of outside coins:</b>")

In [ ]:
img = plt.figure(tight_layout=True, figsize=(8,4))
img.canvas.header_visible = False

gs = gridspec.GridSpec(5, 2)

ax1 = img.add_subplot(gs[0:5, 0])
ax2 = img.add_subplot(gs[1:4, 1])

x = np.linspace(0, 1, 1000)
ax1.fill_between(x, np.sqrt(1 - x**2), np.zeros(len(x)), alpha=0.4)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)

v1 = ax1.scatter([], [], c="b", s=2)
v2 = ax1.scatter([], [], c="r", s=2)

ax2.axhline(Pi, color='green', label="Exact Pi value")
line_pi, = ax2.plot([], [], 'r-', label="Calculated value")
ax2.set_xlabel("Number of coins")
ax2.set_ylabel("Approximate value of $\pi$")
ax2.legend()

def checkbox_change(c):
    ax1.grid(use_quasi.value)
    if use_quasi.value:
        num_coin.min = 25
        num_coin.max = 200
        num_coin.step = 25
        num_coin.value = 25
    else:
        num_coin.min = 1
        num_coin.max = 200
        num_coin.step = 1
        num_coin.value = 1
        
use_quasi.observe(checkbox_change, names='value')
display(label1, label2)
display(use_quasi)
display(HBox([num_coin, run_button, clear_button]))

In [ ]:
def drop_coin(c):
    if use_quasi.value:
        N = int(num_coin.value/25);
        coin = [];
        for i in np.arange(5):
            for j in np.arange(5):
                x = np.random.uniform(i*0.2, (i+1)*0.2, size=N)
                y = np.random.uniform(j*0.2, (j+1)*0.2, size=N)
                coin.extend(np.array((x, y)).T)
        coin = np.array(coin)
    else:
        coin = np.random.random((num_coin.value, 2))

    samples_inside.extend(coin[(coin**2).sum(axis=1) <= 1.])
    samples_outside.extend(coin[(coin**2).sum(axis=1) > 1.])
    
    x1 = np.shape(samples_inside)[0]
    x2 = np.shape(samples_outside)[0]
    
    pi_x.append(x1 + x2)
    pi_y.append(4.0*x1/(x1+x2))
    
    if samples_inside: 
        v1.set_offsets(samples_inside)
    if samples_outside:
        v2.set_offsets(samples_outside)
        
    line_pi.set_data(pi_x, pi_y)
    ax2.set_xlim(pi_x[0], pi_x[-1])
    ax2.set_title(rf"{pi_x[-1]} samples: $\pi \approx {format(pi_y[-1], '.5f')}$")
    label1.value = rf"<b><font color='blue' size=3>Number of inside coins: {str(x1)}</b>"
    label2.value = rf"<b><font color='red' size=3>Number of outside coins: {str(x2)}</b>"


run_button.on_click(drop_coin)

def clear_figure(c):
    global samples_inside, samples_outside, pi_x, pi_y
    v1.set_offsets([[-1, -1]])
    v2.set_offsets([[-1, -1]])
    line_pi.set_data([],[])
    ax2.set_title('')

    samples_inside = [];
    samples_outside = [];

    pi_x = [];
    pi_y = [];
    
    label1.value = rf"<b><font color='blue' size=3>Number of inside coins:</b>"
    label2.value = rf"<b><font color='red' size=3>Number of outside coins:</b>"
    
clear_button.on_click(clear_figure)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Legend
(How to use the interactive visualization)

The left panel shows the simulated square referred to in the theory notebook. The light blue area shows
the quarter circle. When the coin is dropped inside the circle, it will
show up as a blue dot in the figure. Otherwise, it is indicated by a red dot.
The right panel shows the calculated value of Pi as a function of the 
number of dropped coins.
    
You can choose how many coins to drop at each time from the slider. 
Use the Quasi-Monte Carlo method (using quasi-random numbers) by ticking the
checkbox. The "Throw" button is used to throw the coins into the square. You can
clear the simulation by clicking the "Clear" button.